In [2]:
import rlpg.rl_memory as rl_memory
import tqdm
import random
import torch
import numpy as np

In [3]:
buf_size = 10_000
# Writing
buf1 = np.zeros((buf_size,))
mem1 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem1.get_push_idx()
        buf1[idx] = j

buf2 = torch.zeros((buf_size,), requires_grad=False) 
mem2 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem2.get_push_idx()
        buf2[idx] = j

buf3 = [0] * buf_size
mem3 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem3.get_push_idx()
        buf3[idx] = j

buf4 = []
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        buf4.append(j)

print("------------")

# Writing 2
buf1 = np.zeros((10_000,3,50,50))
data = np.zeros((3,50,50))
mem1 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem1.get_push_idx()
        buf1[idx] = data

buf2 = torch.zeros((10_000,3,50,50), requires_grad=False) 
mem2 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem2.get_push_idx()
        buf2[idx] = torch.from_numpy(data)

buf3 = [0] * buf_size
mem3 = rl_memory.CircularBuffer(buf_size)
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        idx = mem3.get_push_idx()
        buf3[idx] = data

buf4 = []
for i in tqdm.tqdm(range(10)):
    for j in range(20_000):
        buf4.append(data)



100%|██████████| 10/10 [00:00<00:00, 1062.68it/s]


------------


100%|██████████| 10/10 [00:00<00:00, 1261.52it/s]


In [3]:
# Acessing
for i in tqdm.tqdm(range(10_000)):
    idxs = mem1.sample_idxs(128)
    buf1[idxs]

for i in tqdm.tqdm(range(10_000)):
    idxs = mem2.sample_idxs(128)
    buf2[idxs]

for i in tqdm.tqdm(range(10_000)):
    random.choices(buf4, k=128)

for i in tqdm.tqdm(range(10_000)):
    random.choices(buf3, k=128)




100%|██████████| 10000/10000 [00:00<00:00, 129890.81it/s]


In [4]:
# to torch
for i in tqdm.tqdm(range(10_000)):
    torch.from_numpy(buf1).requires_grad_(False)

for i in tqdm.tqdm(range(10_000)):
    torch.tensor(buf2, requires_grad=False)

for i in tqdm.tqdm(range(10_000)):
    torch.FloatTensor(buf3, requires_grad=False)

for i in tqdm.tqdm(range(10_000)):
    torch.FloatTensor(buf4, requires_grad=False)

    

  0%|          | 0/10000 [00:00<?, ?it/s]/var/folders/4s/3fzt_vzd12b_22tnj_j6vw7w0000gn/T/ipykernel_78704/2796901395.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(buf2, requires_grad=False)
  0%|          | 0/10000 [00:00<?, ?it/s]


TypeError: new() received an invalid combination of arguments - got (list, requires_grad=bool), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [7]:
buf2.index_add_?

Docstring:
index_add_(dim, index, source, *, alpha=1) -> Tensor

Accumulate the elements of :attr:`alpha` times ``source`` into the :attr:`self`
tensor by adding to the indices in the order given in :attr:`index`. For example,
if ``dim == 0``, ``index[i] == j``, and ``alpha=-1``, then the ``i``\ th row of
``source`` is subtracted from the ``j``\ th row of :attr:`self`.

The :attr:`dim`\ th dimension of ``source`` must have the same size as the
length of :attr:`index` (which must be a vector), and all other dimensions must
match :attr:`self`, or an error will be raised.

For a 3-D tensor the output is given as::

    self[index[i], :, :] += alpha * src[i, :, :]  # if dim == 0
    self[:, index[i], :] += alpha * src[:, i, :]  # if dim == 1
    self[:, :, index[i]] += alpha * src[:, :, i]  # if dim == 2

Note:
    This operation may behave nondeterministically when given tensors on a CUDA device. See :doc:`/notes/randomness` for more information.

Args:
    dim (int): dimension along whic